## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [2]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.26 ms ± 36.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.7 ms ± 65.8 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.8 ms ± 276 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


## 3.2 a)

In [13]:
import cupy as cp
import numpy as np

def grade2_ufunc_cupy(x, y, a, b, c):
    return a*x**2 + b*y + c
    
def gpu_time(func, *args):
    start = cp.cuda.Event()
    end = cp.cuda.Event()

    start.record()
    out = func(*args)
    end.record()

    end.synchronize()
    return out, cp.cuda.get_elapsed_time(start, end)  # ms

size = 5_000_000

# Datos en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)

a = 3.5
b = 2.8
c = 10

# Copiar CPU → GPU
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Medir tiempo
_, t_copy = gpu_time(grade2_ufunc_cupy, a_gpu, b_gpu, a, b, c)

print("Tiempo con arrays copiados CPU→GPU:", t_copy, "ms")

Tiempo con arrays copiados CPU→GPU: 304.3159484863281 ms


In [14]:
# Crear arrays directamente en GPU
a_gpu2 = cp.random.rand(size)
b_gpu2 = cp.random.rand(size)

# Medir tiempo
_, t_nocopy = gpu_time(grade2_ufunc_cupy, a_gpu2, b_gpu2, a, b, c)

print("Tiempo sin copia (arrays creados en GPU):", t_nocopy, "ms")

Tiempo sin copia (arrays creados en GPU): 4.683775901794434 ms


## 3.2 b)

In [18]:
#Creamos la ufunc con vectorize
from numba import vectorize
import numpy as np
import cupy as cp

@vectorize(['float64(float64, float64, float64, float64, float64)'], target='cuda')
def grade2_numba_gpu(x, y, a, b, c):
    return a*x*x + b*y + c
#Para medir el tiempo
def gpu_time(func, *args):
    start = cp.cuda.Event()
    end = cp.cuda.Event()

    start.record()
    out = func(*args)
    end.record()

    end.synchronize()
    return out, cp.cuda.get_elapsed_time(start, end)  

size = 5_000_000

# Datos en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)

a = 3.5
b = 2.8
c = 10

# Medir tiempo (incluye copia CPU→GPU)
_, t_auto_copy = gpu_time(grade2_numba_gpu, a_cpu, b_cpu, a, b, c)

print("Tiempo con copia automática CPU→GPU:", t_auto_copy, "ms")

Tiempo con copia automática CPU→GPU: 60.58553695678711 ms


In [20]:
# Copia manualmente a GPU
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Medir tiempo sin copia
_, t_no_copy = gpu_time(grade2_numba_gpu, a_gpu, b_gpu, a, b, c)

print("Tiempo sin copia (arrays ya en GPU):", t_no_copy, "ms")

Tiempo sin copia (arrays ya en GPU): 4.039135932922363 ms


## 3.2 c)


## Resultados iniciales
### Numba Python: huge improvement, better that numpy code
8.26 ms ± 36.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)

### w/ a numpy ufunc manually coded
18.7 ms ± 65.8 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)

### using the general numpy ufunc 
18.8 ms ± 276 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)

## Empleando Cupy
Tiempo con arrays copiados CPU→GPU: 304.3159484863281 ms
Al emplear CuPy copiando los arrays de la CPU a la GPU tenemos un tiempo alto debido a que la transferencia de CPU a GPU es el proceso más lento. Por otra parte, si no realizamos la copia de los arrays y los creamos directamente en GPU obtenemos el siguiente resultado: Tiempo sin copia (arrays creados en GPU): 4.683775901794434 ms
Un tiempo significativamente menor, dejando claro que CuPy tiene un rendimiento muy alto. Además solo medimos el tiempo que tarda el kernel CUDA en realizar la operación y evitamos el tiempo de copia anterior. 

## Empleando Numba con ufunc y vectorize
Al emplear numba podemos crear ufuncs que se ejecutan en la GPU, pero que se compilan con CUDA. Si pasamos los arrays a la ufunc se realiza la copia CPU→GPU internamente, este paso es que toma más tiempo y nos da el siguiente resultado: Tiempo con copia automática CPU→GPU: 60.58553695678711 ms
En cambio, si antes de llamar a la ufunc realizamos la copia y por tanto no cuantificamos ese tiempo de copia, solo tendremos en cuenta el tiempo de ejecicución del kernel, obteniendo:Tiempo sin copia (arrays ya en GPU): 4.039135932922363 ms

Esto deja claro que la GPU permite una aceleración importante del proceso sabiendo que el paso más costoso sería la copia de datos desde la CPU, si obviamos ese paso el rendimiento de CuPy y ufunc con vectorize es similar. En ambos debemos minimizar